In [2]:
import pandas as pd
import numpy as np 
import requests
import json
import time
from flipside import Flipside
import os

In [3]:
opensea_api_key = os.getenv('opensea_api_key')

***Listing Data***

In [6]:
def fetch_listings(api_key, delay_between_requests=1):
    base_url = "https://api.opensea.io/api/v2/listings/collection/3dns-powered-domains/all"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {"limit": 100} 

    listings = []
    page_count = 0

    while True:
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_listings = data.get("listings", [])
            listings.extend(fetched_listings)
            page_count += 1
            
            # Extract and print the cursor
            next_cursor = data.get("next")
            print(f"Page {page_count}, Cursor: {next_cursor}, Listings Fetched: {len(fetched_listings)}")
            
            if next_cursor:
                params['next'] = next_cursor  # Update the 'next' parameter for the next request
            else:
                break  # No more pages to fetch
                
            # Implementing delay
            time.sleep(delay_between_requests)
            
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break

    print(f"Total pages fetched: {page_count}")
    print(f"Total listings fetched: {len(listings)}")
    
    df = pd.DataFrame(listings)
    return df

***Descriptions***

In [11]:
def save_last_identifier(identifier):
    with open("last_identifier.txt", "w") as file:
        file.write(identifier)

def load_last_identifier():
    try:
        with open("last_identifier.txt", "r") as file:
            return file.read().strip()
    except FileNotFoundError:
        return None

In [12]:
def fetch_all_descriptions(api_key, delay_between_requests=1):
    base_url = "https://api.opensea.io/api/v2/collection/3dns-powered-domains/nfts"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {"limit": 100}

    all_descriptions = []

    page_count = 0
    last_identifier = load_last_identifier()

    while True:
        if last_identifier:
            params['last_identifier'] = last_identifier

        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_descriptions = data.get("nfts", [])
            
            if not fetched_descriptions:
                break

            # Process only name and identifier for each description
            for description in fetched_descriptions:
                processed_description = {
                    "name": description.get('name'),
                    "identifier": description.get('identifier')
                }
                all_descriptions.append(processed_description)
            
            # Update the last_identifier to the latest one fetched
            last_identifier = fetched_descriptions[-1].get('identifier')
            save_last_identifier(last_identifier)
            
            page_count += 1
            next_cursor = data.get("next")
            print(f"Page {page_count}, Cursor: {next_cursor} Descriptions Fetched: {len(fetched_descriptions)}, total fetched: {len(all_descriptions)}")
            
            if next_cursor:
                params['next'] = next_cursor
            else:
                break  # No more pages to fetch

            time.sleep(delay_between_requests)
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break

    print(f"Total pages fetched: {page_count}, Total descriptions fetched: {len(all_descriptions)}")
    
    # Save the processed descriptions to a file
    df = pd.DataFrame(all_descriptions)
    return df

***Events***

In [16]:
import json
import os

def save_last_timestamp(event_type, timestamp):
    data = {}
    if os.path.exists("last_timestamps.json"):
        with open("last_timestamps.json", "r") as file:
            data = json.load(file)
    data[event_type] = timestamp
    with open("last_timestamps.json", "w") as file:
        json.dump(data, file)

def load_last_timestamp(event_type):
    if os.path.exists("last_timestamps.json"):
        with open("last_timestamps.json", "r") as file:
            data = json.load(file)
        return data.get(event_type, None)
    return None

In [20]:
def fetch_event_type(api_key, event_type, all_events, params, headers):
    base_url = f"https://api.opensea.io/api/v2/events/collection/3dns-powered-domains"
    params['event_type'] = event_type
    
    # Load the last timestamp/identifier
    last_timestamp = load_last_timestamp(event_type)
    if last_timestamp:
        params['occurred_after'] = last_timestamp
    
    page_count = 0
    while True:
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_events = data.get("asset_events", [])
            all_events.extend(fetched_events)
            
            if fetched_events:
                # Update the last timestamp/identifier to the latest one fetched
                last_event_time = fetched_events[-1].get("created_date")
                save_last_timestamp(event_type, last_event_time)
            
            page_count += 1
            next_cursor = data.get("next")
            print(f"Fetching {event_type}: Page {page_count}, Events Fetched: {len(fetched_events)}, Total Events: {len(all_events)}, next cursor: {next_cursor}")
            
            if next_cursor:
                params['next'] = next_cursor
            else:
                break  # No more pages to fetch

            time.sleep(1)  # Delay between requests
        else:
            print(f"Failed to fetch {event_type} data: HTTP {response.status_code}, Response: {response.text}")
            break

def fetch_all_events(api_key):
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {
        "limit": 100  # Adjust the limit as needed
    }

    all_events = []

    # Fetch listings
    fetch_event_type(api_key, "listing", all_events, params.copy(), headers)

    # Fetch sales
    fetch_event_type(api_key, "sale", all_events, params.copy(), headers)

    # Save the fetched events to a DataFrame
    print(f"Total events fetched: {len(all_events)}")
    df = pd.DataFrame(all_events)
    return df 

***Data Retrieval/Processing***

In [19]:
events_df = fetch_all_events(api_key= opensea_api_key)

Fetching listing: Page 1, Events Fetched: 50, Total Events: 50, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTI5KzIxJTNBMzQlM0ExNC45ODMyNTcmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM4OTI3NTQ1NDM=
Fetching listing: Page 2, Events Fetched: 50, Total Events: 100, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTI0KzE4JTNBMTIlM0EwMi4xMDU5OTAmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM3NTA4MjcxNDk=
Fetching listing: Page 3, Events Fetched: 50, Total Events: 150, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTIyKzE0JTNBNDklM0EwMC4yMTY2NTAmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM2ODg4MzE0NjY=
Fetching listing: Page 4, Events Fetched: 50, Total Events: 200, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTIyKzAwJTNBMjclM0E1NC4yOTAwNjUmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM2NzIzNDE0NTg=
Fetching listing: Page 5, Events Fetched: 50, Total Events: 250, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTE4KzA3JTNBMjklM0EzNS41MjcwNjEmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM1NzUxMTQ5NjU=
Fetching listing: Page 6, Events Fetched: 50, 

In [13]:
descriptions_df = fetch_all_descriptions(api_key= opensea_api_key)

Page 1, Cursor: LXBrPTE4OTc2NTcxMDE= Descriptions Fetched: 100, total fetched: 100
Page 2, Cursor: LXBrPTE4ODcxMzk4Njg= Descriptions Fetched: 100, total fetched: 200
Page 3, Cursor: LXBrPTE4NzgwMzgyMTY= Descriptions Fetched: 100, total fetched: 300
Page 4, Cursor: LXBrPTE4NjkyNTczMzI= Descriptions Fetched: 100, total fetched: 400
Page 5, Cursor: LXBrPTE4NjIxOTUzMTY= Descriptions Fetched: 100, total fetched: 500
Page 6, Cursor: LXBrPTE4NTA3MDQ1NTY= Descriptions Fetched: 100, total fetched: 600
Page 7, Cursor: LXBrPTE4NDA1MzA2MTM= Descriptions Fetched: 100, total fetched: 700
Page 8, Cursor: LXBrPTE4MzY5NjA1OTc= Descriptions Fetched: 100, total fetched: 800
Page 9, Cursor: LXBrPTE4MzQzNjc3MTM= Descriptions Fetched: 100, total fetched: 900
Page 10, Cursor: LXBrPTE4MzA2OTc0ODk= Descriptions Fetched: 100, total fetched: 1000
Page 11, Cursor: LXBrPTE4Mjc3MTE2ODc= Descriptions Fetched: 100, total fetched: 1100
Page 12, Cursor: LXBrPTE4MjQ4MTAxMzA= Descriptions Fetched: 100, total fetched: 120

In [7]:
listings_df = fetch_listings(api_key= opensea_api_key, delay_between_requests=1)

Page 1, Cursor: cGs9MTc0ODI3MTAyMDYmY3JlYXRlZF9kYXRlPTIwMjQtMDMtMDYrMDMlM0E1MiUzQTMyLjU4ODA4Nw==, Listings Fetched: 100
Page 2, Cursor: cGs9MTg2NDAxMDQxODMmY3JlYXRlZF9kYXRlPTIwMjQtMDQtMjgrMDklM0E1MiUzQTAxLjAwNzE2MQ==, Listings Fetched: 100
Page 3, Cursor: cGs9MTg3OTI1MTI0NjEmY3JlYXRlZF9kYXRlPTIwMjQtMDUtMDQrMTUlM0E1NyUzQTA4LjEzMjQ4Nw==, Listings Fetched: 100
Page 4, Cursor: cGs9MTkzMDU3ODA5MzAmY3JlYXRlZF9kYXRlPTIwMjQtMDUtMjUrMjMlM0E0MCUzQTU2LjI4MzYzNA==, Listings Fetched: 100
Page 5, Cursor: cGs9MTk3MjA5NDcxMDgmY3JlYXRlZF9kYXRlPTIwMjQtMDYtMTMrMDYlM0E1OSUzQTE4LjE4NjQ0MQ==, Listings Fetched: 100
Page 6, Cursor: cGs9MTk5MzMwODgxNTImY3JlYXRlZF9kYXRlPTIwMjQtMDYtMjIrMTUlM0EwNSUzQTA4LjIzMDk0OA==, Listings Fetched: 100
Page 7, Cursor: cGs9MjAyMTc2ODEyNzUmY3JlYXRlZF9kYXRlPTIwMjQtMDctMDQrMTUlM0E0MiUzQTMwLjM1NzA1Nw==, Listings Fetched: 100
Page 8, Cursor: None, Listings Fetched: 4
Total pages fetched: 8
Total listings fetched: 704


In [21]:
listings_df

,order_hash,chain,type,price,protocol_data,protocol_address
0,0x902ff72f57b579d6a2fac92a0714ca5274172ccdeb51...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",{'parameters': {'offerer': '0x64233eaa064ef0d5...,0x00000000000000adc04c56bf30ac9d3c0aaf14dc
1,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",{'parameters': {'offerer': '0x47baba9b83c7cd48...,0x00000000000000adc04c56bf30ac9d3c0aaf14dc
2,0x3040ad3f37193e684cb2baa0811d21cd345108df5a2a...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",{'parameters': {'offerer': '0x97f6f6c5503b9fe3...,0x00000000000000adc04c56bf30ac9d3c0aaf14dc
3,0xb62518c46df32c07046a53f3df61112cc4460bc648ac...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",{'parameters': {'offerer': '0x97f6f6c5503b9fe3...,0x00000000000000adc04c56bf30ac9d3c0aaf14dc
4,0xe4c2e069a35f4b943e449d8e510f952ff3cb2fc133d3...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",{'parameters': {'offerer': '0x99d104c7a9d51953...,0x00000000000000adc04c56bf30ac9d3c0aaf14dc
...,...,...,...,...,...,...
699,0x76a6ca76dd9bdc10d146fffadcb276220828ab704e06...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",{'parameters': {'offerer': '0x22739f9bbc10cf64...,0x0000000000000068f116a894984e2db1123eb395
700,0xe88ff918367aad4e2bca9ed8ae1d6e6ca91f11ad6c21...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",{'parameters': {'offerer': '0x001fbcd3ca9720bc...,0x0000000000000068f116a894984e2db1123eb395
701,0x8a76980edeb088fa648230cf0fcbd36ea9beedb2c924...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",{'parameters': {'offerer': '0x9a93ae4f408a1e7d...,0x0000000000000068f116a894984e2db1123eb395
702,0xc18fc47b36024151d34e664da343db1238ce525e6b63...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",{'parameters': {'offerer': '0x80eea63968fe903d...,0x0000000000000068f116a894984e2db1123eb395


In [22]:
events_df

,event_type,order_hash,order_type,chain,protocol_address,start_date,expiration_date,asset,quantity,maker,taker,payment,criteria,event_timestamp,is_private_listing,closing_date,nft,seller,buyer,transaction
0,order,0x9f368778b2c57cd2d85b80338c2fd16bc29630ab8eb6...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,1.720208e+09,1.735760e+09,{'identifier': '112770027726699050858116509023...,1,0xb05ba164e2e33e7108c106fee5ce06728f7c4bc6,,"{'quantity': '512500000000000000', 'token_addr...",{},1720207913,False,NaN,NaN,NaN,NaN,NaN
1,order,0xc18fc47b36024151d34e664da343db1238ce525e6b63...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,1.720197e+09,1.736095e+09,{'identifier': '131533450042870479718539131055...,1,0x80eea63968fe903da805be04cca7cfb9346b9df2,,"{'quantity': '15000000000000000000', 'token_ad...",{},1720197403,False,NaN,NaN,NaN,NaN,NaN
2,order,0x8a76980edeb088fa648230cf0fcbd36ea9beedb2c924...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,1.720184e+09,1.720443e+09,{'identifier': '283604669650136775214415072033...,1,0x9a93ae4f408a1e7d178650ca9f7ab2135c512eda,,"{'quantity': '5000000000000000000', 'token_add...",{},1720184005,False,NaN,NaN,NaN,NaN,NaN
3,order,0xe88ff918367aad4e2bca9ed8ae1d6e6ca91f11ad6c21...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,1.720176e+09,1.728125e+09,{'identifier': '475343301424942023757896895082...,1,0x001fbcd3ca9720bc35762eabf82821f05d4d3b02,,"{'quantity': '3000000000000000000', 'token_add...",{},1720176098,False,NaN,NaN,NaN,NaN,NaN
4,order,0x773ed587de01c981e2139b33642d6aafe50e6250b3a7...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,1.720157e+09,1.720161e+09,{'identifier': '925705978313878531633041112399...,1,0x70798507b59a73c95acb9e7a037baa54a2250475,,"{'quantity': '1025000000000000000', 'token_add...",{},1720157158,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917,sale,0x8bde44f2d46537cf8d025cc498eaf7bdeadce3631f15...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '1000000000000000', 'token_addres...",NaN,1706970257,NaN,1.706970e+09,{'identifier': '260923785748676206958654735781...,0xcc0e0b9ebd4ce352f5a50b3ae77ea01202c284ee,0xc48a8ac51df7b71e500a5580b89e47ab26ca1472,0x60e952a8e0fa8d052791b2e4a22fd52eb3c759511e62...
2918,sale,0xc0547b5b8d575e3ba16b529ee2530a35c437bc75fa93...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '110000000000000000', 'token_addr...",NaN,1706653465,NaN,1.706653e+09,{'identifier': '566479911085778512201556021651...,0x240ad467a71210629d71d4de22ebde27951c83fc,0x82eb45562f991329ed2867f43fc60f0ba52c3dab,0x6672189deb7829838aabba20ddeb66a3cb5fee5b55a2...
2919,sale,0xff4b9bdd3f140f8211864ef34e0a07ea0a92eaf98f7d...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '10000000000000000', 'token_addre...",NaN,1706203283,NaN,1.706203e+09,{'identifier': '364706086468987072255605061503...,0x7b363822d744143d51c69b0757165c987a6e4ef4,0xe68e8cc7ff772b026c062b9cc28246676d044947,0xd8e71f4d52c43fa770b9aa7ad7d5cfb073f04c7495e0...
2920,sale,0x03188e15aaf9596307117e30355f6b2ac79665fbd1c2...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '150000000000000000', 'token_addr...",NaN,1704080589,NaN,1.704081e+09,{'identifier': '992340878683635895851755157396...,0x31b32020fb1bdf0228c58b80590f07b235f2b0ce,0x64233eaa064ef0d54ff1a963933d0d2d46ab5829,0x186d1ceacf620bab710d11cf928b79e5dd46790ff1d5...


In [23]:
descriptions_df

,name,identifier
0,seismic.box,1013928496915404309227452480946799825762173318...
1,location.box,3538070047554782216243685727791061307869006178...
2,p0rtuga.xyz,1315334500428704797185391310556744981618782458...
3,sprinter.box,6138302165264957769214333741289895695325279240...
4,certificate.box,1366286818048416192675693229694084658125690232...
...,...,...
16327,tech,7961974991147691313806212943424194929272025186...
16328,io,8083426622328183371660078832824394443549950662...
16329,org,7307979787047915423697048548590087477880077770...
16330,net,9704967898254709917779043693758825940947200267...
